In [2]:
from model import ResidualIdenticalBlock
import tensorflow as tf
from model import ResNet

In [ ]:

class ResidualBottleNeckBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, name=None):
        super(ResidualBottleNeckBlock, self).__init__(name=name)
        self.conv1 = tf.keras.layers.Conv2D(
            filters=filters, kernel_size=(1, 1), strides=1, padding="same"
        )
        self.bn1 = tf.keras.layers.BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2D(
            filters=filters, kernel_size=kernel_size, strides=strides, padding="same"
        )
        self.bn2 = tf.keras.layers.BatchNormalization()

        self.conv3 = tf.keras.layers.Conv2D(
            filters=filters * 4, kernel_size=(1, 1), strides=1, padding="same"
        )
        self.bn3 = tf.keras.layers.BatchNormalization()

        if strides != 1:
            self.downsample = tf.keras.layers.Conv2D(
                filters=filters * 4, kernel_size=(3, 3), strides=strides, padding="same"
            )
        else:
            self.downsample = lambda x: x

    def call(self, x):
        residual = self.downsample(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = tf.nn.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        # concat x and residual along the channel axis
        x = tf.concat([x, residual], axis=-1)
        x = tf.nn.relu(x)
        return x


class ResPlainBlock(tf.keras.Model):
    # Name it ResPlainBlock

    def __init__(self, filters, kernel_size, strides=1, name=None):
        super(ResPlainBlock, self).__init__(name=name)

        self.conv1 = tf.keras.layers.Conv2D(
            filters, kernel_size, strides, padding="same"
        )
        self.bn1 = tf.keras.layers.BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, 1, padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()

        if strides != 1:
            self.downsample = tf.keras.layers.Conv2D(filters, 1, strides)
        else:
            self.downsample = lambda x: x

    def call(self, x):
        residual = self.downsample(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)

        # concat x and residual along the channel axis
        x = tf.concat([x, residual], axis=-1)

        x = tf.nn.relu(x)
        return x



In [5]:
filters = 64
# kernel_size = 
rb = ResidualIdenticalBlock(filters, kernel_size=3, strides=1)

In [6]:
inputs = tf.keras.Input(shape=(224, 224, 3))
y, _ = rb(inputs)
model = tf.keras.Model(inputs=inputs, outputs=y)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_2 (Conv2D)              (None, 224, 224, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_2 (BatchNo  (None, 224, 224, 64  256        ['conv2d_2[0][0]']               
 rmalization)                   )                                                           

In [13]:
resnet = ResNet(input_shape=(224, 224, 3), output_shape=10)
resnet.block = ResidualIdenticalBlock
resnet.repeate_block = [2, 2, 2, 2]
model = resnet.build()

ValueError: Exception encountered when calling layer "Block_1" (type Sequential).

Input 0 of layer "SubBlock_0" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(None, 56, 56, 64)

Call arguments received by layer "Block_1" (type Sequential):
  • inputs=tf.Tensor(shape=(None, 56, 56, 64), dtype=float32)
  • training=None
  • mask=None